# 4.4.2.3. Normalized confusion matrix

Effects of the normalization of the confusion matrix on the uncertainty of the results in strongly imbalanced datasets.

Starting from the confusion matrices of the four teams of the MoNuSAC competitions for which the predictions are available (as computed in [Foucart, 2022](http://dx.doi.org/10.13140/RG.2.2.11627.00801))

In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
CLASS_LABELS = ["Epithelial", "Lymphocyte", "Neutrophil", "Macrophage"]

CMs = np.array([
    [
        [0, 1338, 829, 14, 59],
        [831, 6098, 260, 8, 12],
        [507, 79, 7214, 2, 1],
        [8, 5, 39, 118, 2],
        [102, 16, 11, 8, 170]
    ],
    [
        [0, 962, 629, 5, 285],
        [824, 6240, 88, 0, 57],
        [500, 162, 7131, 4, 6],
        [10, 1, 18, 133, 10],
        [115, 16, 1, 11, 164]
    ],
    [
        [0, 2932, 1545, 50, 99],
        [1152, 5960, 96, 0, 1],
        [860, 76, 6864, 3, 0],
        [11, 1, 20, 137, 3],
        [99, 30, 8, 11, 159]
    ],
    [
        [0, 1035, 770, 10, 13],
        [690, 6193, 302, 2, 22],
        [349, 179, 7274, 1, 0],
        [12, 3, 38, 117, 2],
        [90, 30, 7, 25, 155]
    ]
])

print("Classification CMs")
for i, cm in enumerate(CMs):
    print(f"Team {i+1}")
    print(cm[1:, 1:])
    
print("Detection FPs/FNs/TPs")
for i, cm in enumerate(CMs):
    print(f"Team {i+1}")
    print(f"FPs={cm[0, 1:]}")
    print(f"FNs={cm[1:, 0]}")
    print(f"TPs={cm[1:, 1:].sum(axis=1)}")

Classification CMs
Team 1
[[6098  260    8   12]
 [  79 7214    2    1]
 [   5   39  118    2]
 [  16   11    8  170]]
Team 2
[[6240   88    0   57]
 [ 162 7131    4    6]
 [   1   18  133   10]
 [  16    1   11  164]]
Team 3
[[5960   96    0    1]
 [  76 6864    3    0]
 [   1   20  137    3]
 [  30    8   11  159]]
Team 4
[[6193  302    2   22]
 [ 179 7274    1    0]
 [   3   38  117    2]
 [  30    7   25  155]]
Detection FPs/FNs/TPs
Team 1
FPs=[1338  829   14   59]
FNs=[831 507   8 102]
TPs=[6378 7296  164  205]
Team 2
FPs=[962 629   5 285]
FNs=[824 500  10 115]
TPs=[6385 7303  162  192]
Team 3
FPs=[2932 1545   50   99]
FNs=[1152  860   11   99]
TPs=[6057 6943  161  208]
Team 4
FPs=[1035  770   10   13]
FNs=[690 349  12  90]
TPs=[6519 7454  160  217]


The following values are computed:

* $\pi_c$ : the class proportions based on the teams' predictions
* Detection $REC_c$: the RECALL values for each class for the *detection* problem (i.e. an object that is detected is counted regardless of its predicted class)
* Classification $SEN_c$: the SENSITIVITY values for each class for the *classification* problem (i.e. only counting objects which were correctly detected)

We also compute the "error distribution matrix" of the classification task (i.e. if team makes a mistake on a class C sample, where will it be classified?), for the simulation in the next part.

In [3]:
Ns = CMs[0, 1:].sum(axis=1)
pis = Ns/Ns.sum()

print("Number of samples per class in the ground truth:")
for label, n in zip(CLASS_LABELS, Ns):
    print(f"{label} : {n}")
print("Proportion of samples per class in the ground truth:")
for label, pic in zip(CLASS_LABELS, pis):
    print(f"{label} : {pic:.2f}")

rec_c_per_team = []
sen_c_per_team = []
fpr_per_team = []
pis_per_team = []
error_distrib_per_team = []

for cm in CMs:
    rec_cs = np.array([row[1:].sum()/row.sum() for row in cm[1:]])
    rec_c_per_team.append(rec_cs)
    
    sen_c_per_team.append(np.diagonal(cm[1:,1:])/cm[1:,1:].sum(axis=1))
    
    team_pis = cm[:, 1:].sum(axis=0)
    pis_per_team.append(team_pis/team_pis.sum())
    
    team_fpr = cm[0, 1:]/Ns.sum()
    fpr_per_team.append(team_fpr)
    team_error_distrib = np.array([row/row.sum() for row in cm[1:,1:]])
    error_distrib_per_team.append(team_error_distrib)

print("Predicted PI_C")
for idl, label in enumerate(CLASS_LABELS):
    pis_teams = np.array([pis_per_team[idt][idl] for idt in range(4)])
    print(f"{label}: [{pis_teams.min():.2f}, {pis_teams.max():.2f}]")

print("Detection REC_C")
for idl, label in enumerate(CLASS_LABELS):
    rec_teams = np.array([rec_c_per_team[idt][idl] for idt in range(4)])
    print(f"{label}: [{rec_teams.min():.2f}, {rec_teams.max():.2f}]")
    
print("Classification SEN_C")
for idl, label in enumerate(CLASS_LABELS):
    sen_teams = np.array([sen_c_per_team[idt][idl] for idt in range(4)])
    print(f"{label}: [{sen_teams.min():.2f}, {sen_teams.max():.2f}]")

Number of samples per class in the ground truth:
Epithelial : 7209
Lymphocyte : 7803
Neutrophil : 172
Macrophage : 307
Proportion of samples per class in the ground truth:
Epithelial : 0.47
Lymphocyte : 0.50
Neutrophil : 0.01
Macrophage : 0.02
Predicted PI_C
Epithelial: [0.46, 0.50]
Lymphocyte: [0.47, 0.52]
Neutrophil: [0.01, 0.01]
Macrophage: [0.01, 0.03]
Detection REC_C
Epithelial: [0.84, 0.90]
Lymphocyte: [0.89, 0.96]
Neutrophil: [0.93, 0.95]
Macrophage: [0.63, 0.71]
Classification SEN_C
Epithelial: [0.95, 0.98]
Lymphocyte: [0.98, 0.99]
Neutrophil: [0.72, 0.85]
Macrophage: [0.71, 0.85]


Computing all the classification metrics (looking only at detected samples, so it's a "4 class" problem, not a "4 class + background" problem), based on the CM and the NCM:

In [4]:
from metrics.classification import GM, MCC, Accuracy, kappaU, hF1, sF1
from metrics.unbalance import rescale_metric
from functools import partial

metrics = {
    'Acc ': Accuracy, 
    'GM  ': GM,
    'MCCn': partial(rescale_metric, MCC), 
    'hF1 ': hF1, 
    'sF1 ': sF1, 
    'kUn ': partial(rescale_metric, kappaU)
}

res_per_team = np.zeros((4,len(metrics)*2))

#print("CM")
for idt in range(4):
    #print(f'Team {idt+1}')
    for idm,(key,metric) in enumerate(metrics.items()):
        res = metric(CMs[idt,1:,1:])
        res_per_team[idt][idm] = res
        #print(f'{key}:\t {res:.3f}')
        
#print("NCM")
for idt in range(4):
    #print(f'Team {idt+1}')
    NCM = CMs[idt, 1:, 1:]
    NCM = np.array([row/row.sum() for row in NCM])
    for idm,(key,metric) in enumerate(metrics.items()):
        res = metric(NCM)
        res_per_team[idt][len(metrics)+idm] = res
        #print(f'{key}:\t {metric(NCM):.3f}')
        
print(f"Team    {'   '.join(list(metrics.keys()))} ||  {'   '.join(list(metrics.keys()))}")
for idt in range(4):
    res = [f'{r:.3f}' for r in res_per_team[idt]]
    print(f"Team {idt+1}  {'  '.join(res)}")

Team    Acc    GM     MCCn   hF1    sF1    kUn  ||  Acc    GM     MCCn   hF1    sF1    kUn 
Team 1  0.968  0.867  0.970  0.902  0.972  0.970  0.873  0.867  0.919  0.883  0.887  0.916
Team 2  0.973  0.904  0.975  0.897  0.978  0.975  0.907  0.904  0.939  0.908  0.931  0.938
Team 3  0.981  0.892  0.982  0.928  0.984  0.982  0.897  0.892  0.933  0.901  0.915  0.931
Team 4  0.957  0.834  0.959  0.870  0.962  0.959  0.843  0.834  0.899  0.851  0.872  0.895


**Simulation results**

* Create "fake" dataset of N randomly sampled examples based on the MoNuSAC distribution.
* Recompute the "confusion matrices" using the recall and sensitivities
* Recompute all the metrics
* Repeat 1.000 times and look at how much the metrics can change due to random sampling

The reported results are the "uncertainty" due to the random sampling, computed as the maximum divergence across the four teams, with the divergence the percentile 97.5 - percentile 2.5 of the results.

In [5]:
# Random sampling based on dataset pis:
def get_sample(N):
    samples = []
    draw = np.random.random((N,))
    classes = np.zeros_like(draw)
    cumul = 0
    for c,pi in enumerate(pis):
        selected = draw <= pi+cumul
        draw[selected] = 1.5
        classes[selected] = c
        cumul += pi
    
    return np.array([(classes==c).sum() for c in range(4)])

In [6]:
from tqdm import tqdm

n_exp = 5_000

for N in [15_000, 5_000, 1_000]:
    print(f"{N} samples")

    cm_metrics_per_team = np.zeros((4, len(metrics), n_exp))
    ncm_metrics_per_team = np.zeros((4, len(metrics), n_exp))

    for exp in tqdm(range(n_exp)):
        N_sample = get_sample(N)
        for idt in range(4):
            cm = np.zeros((4,4)).astype('int')
            detected = np.round(N_sample*rec_c_per_team[idt]).astype('int')
            for c in range(4):
                cm[c] = np.round(detected[c]*error_distrib_per_team[idt][c])
            ncm = np.array([row/row.sum() for row in cm])
            for idm,(key,metric) in enumerate(metrics.items()):
                cm_metrics_per_team[idt, idm, exp] = metric(cm)
                ncm_metrics_per_team[idt, idm, exp] = metric(ncm)
    
    cm_args = cm_metrics_per_team.argsort(axis=2)
    ncm_args = ncm_metrics_per_team.argsort(axis=2)

    pmin = int(n_exp*0.975)
    pmax = int(n_exp*0.025)

    for idm, metric in enumerate(metrics):
        max_cm = 0
        max_ncm = 0
        for idt in range(4):
            sorted_team_cm = cm_metrics_per_team[idt, idm, cm_args[idt, idm]]
            team_uncertainty_cm = sorted_team_cm[pmin]-sorted_team_cm[pmax]
            max_cm = max(team_uncertainty_cm, max_cm)
            sorted_team_ncm = ncm_metrics_per_team[idt, idm, ncm_args[idt, idm]]
            team_uncertainty_ncm = sorted_team_ncm[pmin]-sorted_team_ncm[pmax]
            max_ncm = max(team_uncertainty_ncm, max_ncm)
        print(f"{metric}: CM = {max_cm:.3f}, NCM = {max_ncm:.3f}")

15000 samples


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:24<00:00, 204.75it/s]


Acc : CM = 0.001, NCM = 0.003
GM  : CM = 0.003, NCM = 0.003
MCCn: CM = 0.001, NCM = 0.002
hF1 : CM = 0.006, NCM = 0.003
sF1 : CM = 0.001, NCM = 0.002
kUn : CM = 0.001, NCM = 0.002
5000 samples


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:21<00:00, 229.89it/s]


Acc : CM = 0.002, NCM = 0.010
GM  : CM = 0.011, NCM = 0.011
MCCn: CM = 0.002, NCM = 0.006
hF1 : CM = 0.011, NCM = 0.009
sF1 : CM = 0.001, NCM = 0.007
kUn : CM = 0.002, NCM = 0.006
1000 samples


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:19<00:00, 253.54it/s]


Acc : CM = 0.006, NCM = 0.052
GM  : CM = 0.056, NCM = 0.056
MCCn: CM = 0.005, NCM = 0.035
hF1 : CM = 0.046, NCM = 0.054
sF1 : CM = 0.003, NCM = 0.035
kUn : CM = 0.005, NCM = 0.034
